In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv
/kaggle/input/icr-identify-age-related-conditions/greeks.csv
/kaggle/input/icr-identify-age-related-conditions/train.csv
/kaggle/input/icr-identify-age-related-conditions/test.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
# !pip install optuna
import optuna

# Data preprocessing (scaling the features)
from sklearn.preprocessing import StandardScaler

# Read the data
train_df = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
test_df = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
greeks_df = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')

train_df["EJ"] = train_df["EJ"].replace({"A": 0, "B": 1})
test_df["EJ"] = test_df["EJ"].replace({"A": 0, "B": 1})

train_df.fillna(train_df.mean(), inplace=True)
test_df.fillna(test_df.mean(), inplace=True)

train_values = train_df.drop(["Class", "Id"], axis=1)
test_values = train_df["Class"]

# Split the data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_values, test_values, test_size=0.2, random_state=73)

# Data preprocessing: Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Define the neural network model with L2 regularization and dropout
def build_model(num_units_layer1, num_units_layer2, learning_rate, dropout_rate):
    model = Sequential()
    model.add(Dense(units=num_units_layer1, activation='relu', input_shape=(train_values.shape[1],),
                    kernel_regularizer='l2'))  # L2 regularization
    model.add(BatchNormalization())  # Batch Normalization layer
    model.add(Dropout(dropout_rate))  # Dropout layer for regularization
    model.add(Dense(units=num_units_layer2, activation='relu', kernel_regularizer='l2'))  # L2 regularization
    model.add(BatchNormalization())  # Batch Normalization layer
    model.add(Dropout(dropout_rate))  # Dropout layer for regularization
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=learning_rate))

    return model

# Objective function for Hyperparameter Optimization (using Optuna)
def objective(trial):
    num_units_layer1 = trial.suggest_int("num_units_layer1", 32, 128)
    num_units_layer2 = trial.suggest_int("num_units_layer2", 16, 64)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2)
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5)

    model = build_model(num_units_layer1, num_units_layer2, learning_rate, dropout_rate)

    # Early stopping callback to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Learning rate scheduling with patience=3 (reduce learning rate if validation loss plateaus for 3 epochs)
    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3)

    # Train the model
    history = model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_val, y_val),
                        callbacks=[early_stopping, lr_scheduler], verbose=0)

    # Make predictions
    final_pred = model.predict(X_val)

    # Calculate log loss
    loss = log_loss(y_val, final_pred)
    return loss

# Call the objective function for hyperparameter tuning
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

# Retrieve best hyperparameters
best_params = study.best_params

# Create an ensemble of models with L2 regularization and dropout
num_models = 10
models = []
for i in range(num_models):
    model = build_model(best_params["num_units_layer1"], best_params["num_units_layer2"],
                        best_params["learning_rate"], best_params["dropout_rate"])
    models.append(model)

# Train the models on the entire dataset
for model in models:
    model.fit(X_train, y_train, batch_size=32, epochs=100, verbose=1)

# Make predictions on the test data and average the results using weighted average based on validation performance
val_preds = [model.predict(X_val) for model in models]
val_losses = [log_loss(y_val, pred) for pred in val_preds]
weights = [1.0 / loss for loss in val_losses]  # Inverse of validation loss as weights
weights_sum = sum(weights)
final_pred_ensemble = np.zeros_like(val_preds[0])

for i in range(num_models):
    final_pred_ensemble += val_preds[i] * (weights[i] / weights_sum)

# Make predictions on the test data using the weighted average of the models
test_values_scaled = scaler.transform(test_df.drop("Id", axis=1))
test_preds = [model.predict(test_values_scaled) for model in models]
final_pred_test = np.zeros_like(test_preds[0])

for i in range(num_models):
    final_pred_test += test_preds[i] * (weights[i] / weights_sum)

# Prepare the submission
sample_submission_df = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv')
sample_submission_df['Id'] = test_df.reset_index()['Id']
sample_submission_df["class_0"] = 1 - final_pred_test.flatten()  # Assuming class_0 is the positive class
sample_submission_df["class_1"] = final_pred_test.flatten()      # Assuming class_1 is the negative class
sample_submission_df.set_index('Id').to_csv('submission.csv')


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

4/4 [==============================] - 0s 2ms/step


[I 2023-07-20 14:05:24,195] Trial 0 finished with value: 0.37612669809311017 and parameters: {'num_units_layer1': 79, 'num_units_layer2': 50, 'learning_rate': 0.008012238952084676, 'dropout_rate': 0.4514525795507893}. Best is trial 0 with value: 0.37612669809311017.


4/4 [==============================] - 0s 2ms/step


[I 2023-07-20 14:05:27,299] Trial 1 finished with value: 0.27934503086750023 and parameters: {'num_units_layer1': 97, 'num_units_layer2': 22, 'learning_rate': 0.009374399732980473, 'dropout_rate': 0.3313571774571368}. Best is trial 1 with value: 0.27934503086750023.


...

4/4 [==============================] - 0s 2ms/step


[I 2023-07-20 14:09:24,256] Trial 98 finished with value: 0.375243140522661 and parameters: {'num_units_layer1': 62, 'num_units_layer2': 21, 'learning_rate': 0.004621505364863438, 'dropout_rate': 0.498137247477919}. Best is trial 7 with value: 0.2606252274634733.


4/4 [==============================] - 0s 2ms/step


[I 2023-07-20 14:09:27,811] Trial 99 finished with value: 0.33788637293996476 and parameters: {'num_units_layer1': 53, 'num_units_layer2': 61, 'learning_rate': 0.0053571649426193465, 'dropout_rate': 0.4707768872501595}. Best is trial 7 with value: 0.2606252274634733.


Epoch 1/100
16/16 [==============================] - 1s 2ms/step - loss: 1.8827
Epoch 2/100
16/16 [==============================] - 0s 2ms/step - loss: 1.4672
Epoch 3/100
16/16 [==============================] - 0s 2ms/step - loss: 1.2867
Epoch 4/100
16/16 [==============================] - 0s 2ms/step - loss: 1.1272
Epoch 5/100
16/16 [==============================] - 0s 2ms/step - loss: 1.0231
Epoch 6/100
16/16 [==============================] - 0s 2ms/step - loss: 0.9274
Epoch 7/100
16/16 [==============================] - 0s 2ms/step - loss: 0.8300
Epoch 8/100
16/16 [==============================] - 0s 2ms/step - loss: 0.7598
Epoch 9/100
16/16 [==============================] - 0s 2ms/step - loss: 0.7013
Epoch 10/100
16/16 [==============================] - 0s 2ms/step - loss: 0.6306
Epoch 11/100
16/16 [==============================] - 0s 3ms/step - loss: 0.6220
Epoch 12/100
16/16 [==============================] - 0s 2ms/step - loss: 0.5420
Epoch 13/100
16/16 [=================